# CVAE
Dataset

In [ ]:
import os, glob, numpy as np, torch
from PIL import Image
import torch
from torch.utils.data import Dataset
from torchvision import transforms
from torch.nn import Module, Sequential, Conv2d, Dropout2d, BatchNorm2d, Flatten, Linear, ConvTranspose2d, ReLU, Tanh, \
    LeakyReLU, init
import argparse
import time
from datetime import datetime
from matplotlib import pyplot as plt
from torch import device, cuda, save, load, no_grad, cat, clamp, randn
from torch.nn.functional import mse_loss
from torch.nn.utils import clip_grad_norm_
from torch.optim import AdamW
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader
from torchvision.utils import make_grid
from tqdm import tqdm

from dataset import CaptionImageSet
from model import ConvCVAE

SUPPORTED_IMG_EXTENSIONS = (".jpg", ".jpeg", ".png", ".webp")
np.random.seed(2)
torch.manual_seed(2)

device = device("cuda" if cuda.is_available() else "cpu")
print(f"Using device: {device}")

In [ ]:
class CaptionImageSet(Dataset):
    """Dataset class created to load images and its embeddings"""

    def __init__(self, root='data/processed', img_size=64, test_split=0.1, is_test=False):
        super(CaptionImageSet,self).__init__()
        self.img_dir = os.path.join(root, "images")
        self.emb_dir = os.path.join(root, "embeds")
        self.img_size = img_size
        self.ids = []
        self.test_split = test_split
        self.is_test = is_test
        if is_test:
            self.preprocess = transforms.Compose([
                transforms.Resize((self.img_size, self.img_size)),
                transforms.RandomHorizontalFlip(),
                transforms.ToTensor(),
                transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])])
        else:
            self.preprocess = transforms.Compose([
                transforms.Resize((img_size, img_size)),
                transforms.ToTensor(),
                transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
            ])

        if not os.path.exists(self.img_dir):
            raise ValueError(f"Image directory not found: {self.img_dir}")
        if not os.path.exists(self.emb_dir):
            raise ValueError(f"Embedding directory not found: {self.emb_dir}")

        self._load_ids()
        self._split_data()

        print(f"Loaded {'test' if is_test else 'train'} dataset: {len(self.ids)} samples")

    def _load_ids(self):
        for file_name in glob.glob(os.path.join(self.img_dir, "*")):
            base_name, extension_name = os.path.splitext(os.path.basename(file_name))
            if extension_name.lower() in SUPPORTED_IMG_EXTENSIONS and os.path.exists(
                    os.path.join(self.emb_dir, base_name + ".npy")):
                self.ids.append(base_name)
        self.ids.sort()
        print(f"Loaded {len(self.ids)} valid pairs of images and embeddings")

    def _split_data(self):
        if self.test_split > 0:
            n_test = int(len(self.ids) * self.test_split)
            if self.is_test:
                self.ids = self.ids[:n_test]
            else:
                self.ids = self.ids[n_test:]

    def __len__(self):
        return len(self.ids)

    def __getitem__(self, idx):
        id = self.ids[idx]

        img_path = None
        img_base_path = os.path.join(self.img_dir, id)
        for ext in SUPPORTED_IMG_EXTENSIONS:
            if os.path.exists(img_base_path + ext):
                img_path = img_base_path + ext
                break

        if img_path is None:
            raise ValueError(f"Image path {id} not found")

        img = Image.open(img_path).convert('RGB')

        embedding = np.load(os.path.join(self.emb_dir, id + ".npy")).astype(np.float32)
        if embedding.ndim > 1:
            embedding = embedding.flatten()

        return self.preprocess(img), torch.from_numpy(embedding)



## Model


In [ ]:
class Encoder(Module):
    '''Encoder component'''
    def __init__(self, latent_dim, img_dim):
        super(Encoder, self).__init__()
        self.latent_dim = latent_dim
        self.img_dim = img_dim
        self.encoder_input = self.img_dim[-1] + 512
        self.layers = Sequential(
            Conv2d(self.encoder_input, 64, kernel_size=4, stride=2, padding=1),
            LeakyReLU(0.2),
            Dropout2d(0.1),

            Conv2d(64, 128, kernel_size=4, stride=2, padding=1),
            BatchNorm2d(128),
            LeakyReLU(0.2),
            Dropout2d(0.1),

            Conv2d(128, 256, kernel_size=4, stride=2, padding=1),
            BatchNorm2d(256),
            LeakyReLU(0.2),
            Dropout2d(0.1),

            Conv2d(256, 512, kernel_size=4, stride=2, padding=1),
            BatchNorm2d(512),
            LeakyReLU(0.2),
            Flatten(),
        )

        self.mean = Linear(512 * 4 * 4, latent_dim)
        self.log_of_variance = Linear(512 * 4 * 4, latent_dim)


    def forward(self, img, embedding):
        embedding = embedding.view(embedding.size(0), 512, 1, 1).repeat(1, 1, self.img_dim[0], self.img_dim[1])
        result = cat((img, embedding), dim=1)
        x = self.layers(result)
        mean = self.mean(x)
        log_of_var = clamp(self.log_of_variance(x), min=-10, max=10)
        return mean, log_of_var

In [ ]:
class Decoder(Module):
    """Decoder component"""
    def __init__(self, latent_dim):
        super(Decoder, self).__init__()
        self.decoder_input = Linear(latent_dim + 512, 512 * 4 * 4)
        self.layers = Sequential(
            ConvTranspose2d(512, 256, kernel_size=4, stride=2, padding=1),
            BatchNorm2d(256),
            ReLU(),

            ConvTranspose2d(256, 128, kernel_size=4, stride=2, padding=1),
            BatchNorm2d(128),
            ReLU(),

            ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1),
            BatchNorm2d(64),
            ReLU(),

            ConvTranspose2d(64, 3, kernel_size=4, stride=2, padding=1),
            Tanh()
        )

    def forward(self, z, clip_embedding):
        z = cat([z, clip_embedding], dim=1)
        x = self.decoder_input(z)
        x = x.view(-1, 512, 4, 4)
        x = self.layers(x)
        return x


In [ ]:
class ConvCVAE(Module):
    """Conditional Variational Autoencoder."""

    def __init__(self, img_dim=[64, 64, 3], latent_dim=128):
        super(ConvCVAE, self).__init__()
        self.img_size = img_dim[0]
        self.latent_dim = latent_dim

        self.encoder = Encoder(img_dim=img_dim,latent_dim=latent_dim)

        self.decoder = Decoder(latent_dim=latent_dim)

        self.apply(self._init_weights)

    def _init_weights(self, m):
        if isinstance(m, (Conv2d, ConvTranspose2d)):
            init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='leaky_relu')
            if m.bias is not None:
                init.constant_(m.bias, 0)
        elif isinstance(m, BatchNorm2d):
            init.constant_(m.weight, 1)
            init.constant_(m.bias, 0)
        elif isinstance(m, Linear):
            init.xavier_normal_(m.weight)
            init.constant_(m.bias, 0)

    def reparameterize(self, mean, log_var):
        std = torch.exp(0.5 * log_var)
        e = torch.randn_like(std)
        z = mean + e * std
        return z

    def encode(self, x, embedding):
        return self.encoder(x, embedding)

    def decode(self, z, embedding):
        return self.decoder(z, embedding)

    def forward(self, x, embedding):
        mean, log_var = self.encode(x, embedding)
        z = self.reparameterize(mean, log_var)
        recon_x = self.decode(z, embedding)

        return recon_x, mean, log_var

    def generate(self, embedding, num_samples=1):
        embedding = embedding.unsqueeze(0).repeat(num_samples, 1) if len(embedding.shape) == 1 else embedding

        z = torch.randn(embedding.size(0), self.latent_dim).to(embedding.device)

        with torch.no_grad():
            generated = self.decode(z, embedding)

        return generated

    def reconstruct(self, x, clip_embedding):
        with torch.no_grad():
            mean, _ = self.encode(x, clip_embedding)
            recon = self.decode(mean, clip_embedding)
        return recon


In [ ]:

def load_dataset(args):
    print("\n" + "=" * 70)
    print("LOADING DATASET")
    print("=" * 70)

    train_dataset = CaptionImageSet(
        root=args.data_root,
        img_size=args.img_size,
        test_split=args.test_split,
        is_test=False
    )

    test_dataset = CaptionImageSet(
        root=args.data_root,
        img_size=args.img_size,
        test_split=args.test_split,
        is_test=True
    )

    train_loader = DataLoader(
        train_dataset,
        batch_size=args.batch_size,
        shuffle=True,
        num_workers=2,
        pin_memory=True
    )

    test_loader = DataLoader(
        test_dataset,
        batch_size=args.batch_size,
        shuffle=False,
        num_workers=2,
        pin_memory=True
    )

    return train_loader, test_loader


def generate_img_grid(images, save_path, title):
    images = clamp((images * 0.5 + 0.5), 0, 1)
    grid = make_grid(images, nrow=4)
    np_grid = grid.cpu().numpy()

    plt.figure(figsize=(10, 10))
    plt.imshow(np.transpose(np_grid, (1, 2, 0)))
    plt.axis('off')
    plt.title(title)
    plt.savefig(save_path)
    plt.close()


def loss_function(recon_x, x, mean, log_var, beta=1.0, warmup_factor=1.0):
    recon_loss = mse_loss(recon_x, x, reduction='sum')
    kl_loss = clamp(-0.5 * torch.sum(1 + log_var - mean.pow(2) - log_var.exp()), max=1e6)
    total_loss = recon_loss + (beta * warmup_factor * kl_loss)

    return total_loss, recon_loss, kl_loss


def get_warmup_factor(epoch, warmup_epochs=10):
    if epoch < warmup_epochs:
        return epoch / warmup_epochs
    return 1.0


def train_epoch(model, train_loader, optimizer, beta, epoch, warmup_epochs):
    model.train()
    tr_loss, tr_recon_loss, tr_kl = 0, 0, 0

    warmup_factor = get_warmup_factor(epoch, warmup_epochs)

    for batch_idx, (data, clip_emb) in enumerate(tqdm(train_loader)):
        data = data.to(device)
        clip_emb = clip_emb.to(device)

        optimizer.zero_grad()
        recon_batch, mean, log_var = model(data, clip_emb)
        loss, recon_loss, kl_loss = loss_function(recon_batch, data, mean, log_var, beta, warmup_factor)

        loss.backward()
        clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()

        tr_loss += loss.item()
        tr_recon_loss += recon_loss.item()
        tr_kl += kl_loss.item()

    n_samples = len(train_loader.dataset)
    return tr_loss / n_samples, tr_recon_loss / n_samples, tr_kl / n_samples, warmup_factor


def validate(model, val_loader, beta):
    model.eval()
    val_loss, val_recon, val_kl = 0, 0, 0

    with no_grad():
        for data, clip_emb in val_loader:
            data = data.to(device)
            clip_emb = clip_emb.to(device)
            recon_batch, mean, log_var = model(data, clip_emb)
            loss, recon_loss, kl_loss = loss_function(recon_batch, data, mean, log_var, beta, warmup_factor=1.0)
            val_loss += loss.item()
            val_recon += recon_loss.item()
            val_kl += kl_loss.item()

    n_samples = len(val_loader.dataset)
    return val_loss / n_samples, val_recon / n_samples, val_kl / n_samples


def train(args):

    def save_plots_images(label):
        data_iter = iter(test_loader)
        images, embeddings = next(data_iter)

        model.eval()
        with no_grad():
            images = images.to(device)
            embeddings = embeddings.to(device)
            recon, _, _ = model(images, embeddings)

        comparison = cat([images[:8].cpu(), recon[:8].cpu()])
        save_path = os.path.join(result_dir, f"reconstruction_epoch_{label}.png")
        generate_img_grid(comparison, save_path, f"Reconstruction - Epoch {label}")

        generated = model.generate(embeddings[:16], num_samples=16)
        save_path = os.path.join(result_dir, f"generated_epoch_{label}.png")
        generate_img_grid(generated, save_path, f"Generated - Epoch {label}")

        plot_losses(history, os.path.join(loss_dir, f"losses_epoch_{label}.png"))

    start_time_total = time.perf_counter()
    dt = datetime.now()
    dt_str = dt.strftime("%Y-%m-%d_%H-%M-%S")

    output_root = f'{dt_str}_{args.output_dir}'
    os.makedirs(output_root, exist_ok=True)

    checkpoint_dir = os.path.join(output_root, "checkpoints")
    os.makedirs(checkpoint_dir, exist_ok=True)

    result_dir = os.path.join(output_root, "results")
    os.makedirs(result_dir, exist_ok=True)

    loss_dir = os.path.join(output_root, "losses")
    os.makedirs(loss_dir, exist_ok=True)

    train_loader, test_loader = load_dataset(args)

    model = ConvCVAE(img_dim=[args.img_size, args.img_size, 3],latent_dim=args.latent_dim).to(device)

    optimizer = AdamW(model.parameters(), lr=args.lr, weight_decay=1e-5)

    scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5)

    print(f"\nModel Architecture:")
    print(f"  Latent dim: {args.latent_dim}")
    print(f"  CLIP dim: 512")
    print(f"  Image size: {args.img_size}")
    print(f"  Beta: {args.beta}")
    print(f"  KL warmup epochs: {args.warmup_epochs}")

    epoch_idx = 0
    min_loss = float('inf')

    if args.checkpoint:
        checkpoint = load(args.checkpoint, map_location=device)
        epoch_idx = checkpoint['epoch_idx'] + 1
        model.load_state_dict(checkpoint['model_state'])
        optimizer.load_state_dict(checkpoint['optimizer_state'])
        min_loss = checkpoint.get('min_loss', float('inf'))
        print(f"\nLoaded checkpoint from epoch {checkpoint['epoch']}")

    print("\n" + "=" * 70)
    print("STARTING TRAINING")
    print("=" * 70)
    history = {
        'train_loss': [],
        'train_recon': [],
        'train_kl': [],
        'val_loss': [],
        'val_recon': [],
        'val_kl': []
    }

    for epoch in range(epoch_idx, args.epochs):
        print(f"\nEpoch {epoch + 1}/{args.epochs}")

        train_loss, train_recon, train_kl, warmup = train_epoch(
            model, train_loader, optimizer, args.beta, epoch, args.warmup_epochs
        )

        val_loss, val_recon, val_kl = validate(model, test_loader, args.beta)

        scheduler.step(val_loss)

        history['train_loss'].append(train_loss)
        history['train_recon'].append(train_recon)
        history['train_kl'].append(train_kl)
        history['val_loss'].append(val_loss)
        history['val_recon'].append(val_recon)
        history['val_kl'].append(val_kl)

        print(f"Warmup: {warmup:.2f} | LR: {optimizer.param_groups[0]['lr']:.6f}")
        print(f"Train - Loss: {train_loss:.4f}, Recon: {train_recon:.4f}, KL: {train_kl:.4f}")
        print(f"Val   - Loss: {val_loss:.4f}, Recon: {val_recon:.4f}, KL: {val_kl:.4f}")

        # Save best model
        if val_loss < min_loss:
            min_loss = val_loss
            save({
                'epoch_idx': epoch,
                'model_state': model.state_dict(),
                'optimizer_state': optimizer.state_dict(),
                'train_loss': train_loss,
                'val_loss': val_loss,
                'min_loss': min_loss,
            }, os.path.join(checkpoint_dir, "best_checkpoint.pt"))
            print ('Best checkpoint saved')
            save_plots_images('best')


        if (epoch + 1) % args.save_interval == 0:
            checkpoint_path = os.path.join(checkpoint_dir, f"model_epoch_{epoch + 1}.pt")
            save({
                'epoch_idx': epoch,
                'model_state': model.state_dict(),
                'optimizer_state': optimizer.state_dict(),
                'train_loss': train_loss,
                'val_loss': val_loss,
                'min_loss': min_loss,
            }, checkpoint_path)
            print(f"Checkpoint saved: {checkpoint_path}")

            save_plots_images(str(epoch + 1))

    final_path = os.path.join(checkpoint_dir, "model_final.pt")
    save({
        'epoch_idx': args.epochs - 1,
        'model_state': model.state_dict(),
        'optimizer_state': optimizer.state_dict(),
        'min_loss': min_loss,
    }, final_path)

    plot_losses(history, os.path.join(loss_dir, "losses_final.png"))

    total_time = time.perf_counter() - start_time_total
    print(f"\n" + "=" * 70)
    print(f"TRAINING COMPLETE")
    print(f"Best validation loss: {min_loss:.4f}")
    print(f"Total time: {total_time:.2f}s ({total_time / 60:.2f} minutes)")
    print(f"Results saved to: {output_root}")
    print("=" * 70)




def plot_losses(history, save_path):
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))

    epochs = range(1, len(history['train_loss']) + 1)

    axes[0].plot(epochs, history['train_loss'], 'b-', label='Train')
    axes[0].plot(epochs, history['val_loss'], 'r-', label='Val')
    axes[0].set_title('Total Loss')
    axes[0].set_xlabel('Epoch')
    axes[0].set_ylabel('Loss')
    axes[0].legend()
    axes[0].grid(True)

    axes[1].plot(epochs, history['train_recon'], 'b-', label='Train')
    axes[1].plot(epochs, history['val_recon'], 'r-', label='Val')
    axes[1].set_title('Reconstruction Loss')
    axes[1].set_xlabel('Epoch')
    axes[1].set_ylabel('Loss')
    axes[1].legend()
    axes[1].grid(True)

    axes[2].plot(epochs, history['train_kl'], 'b-', label='Train')
    axes[2].plot(epochs, history['val_kl'], 'r-', label='Val')
    axes[2].set_title('KL Divergence')
    axes[2].set_xlabel('Epoch')
    axes[2].set_ylabel('Loss')
    axes[2].legend()
    axes[2].grid(True)

    plt.tight_layout()
    plt.savefig(save_path, bbox_inches='tight', dpi=150)
    plt.close()


In [ ]:
parser = argparse.ArgumentParser(description="Train CLIP-CVAE on food images")
parser.add_argument("--data_root", type=str, default="data/processed",
                    help="Root directory containing images and embeds folders")
parser.add_argument("--output_dir", type=str, default="results_pytorch",
                    help="Output directory for checkpoints and samples")
parser.add_argument("--img_size", type=int, default=64,
                    help="Image size (height and width)")
parser.add_argument("--batch_size", type=int, default=32,
                    help="Batch size for training")
parser.add_argument("--test_split", type=float, default=0.1,
                    help="Fraction of data to use for testing")
parser.add_argument("--latent_dim", type=int, default=128,
                    help="Latent dimension size")
parser.add_argument("--beta", type=float, default=0.5,
                    help="Beta parameter for KL loss weighting")
parser.add_argument("--lr", type=float, default=2e-4,
                    help="Learning rate")
parser.add_argument("--epochs", type=int, default=100,
                    help="Number of training epochs")
parser.add_argument("--warmup_epochs", type=int, default=10,
                    help="Number of epochs for KL warmup")
parser.add_argument("--save_interval", type=int, default=5,
                    help="Save checkpoint every N epochs")
parser.add_argument("--checkpoint", type=str, default=None,
                    help="Path to checkpoint to resume training")

args = parser.parse_args()
train(args)